## Tim's Models

Consists of: CatBoost, Random Forests, Linear Regressor, K-Nearest-Neighbour.

In [1]:
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
def normalize_data(df):
    x = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)

In [3]:
# Load data
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')

# Check data
print("Training set ", train.shape)
print("Test set ", test.shape)

Training set  (3000, 3737)
Test set  (4398, 5047)


In [4]:
# Fill remaining NA's with 0 and negatives with 0
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
train[train < 0] = 0
test[test < 0] = 0

In [5]:
# Drop ID Column
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

In [6]:
train.describe()

,budget,popularity,runtime,revenue,name_collection,genre_Drama,genre_Comedy,genre_Thriller,genre_Action,genre_Romance,...,prodc_Quick Six Entertainment,production_countries_count,release_month,release_day,release_year,spoken_languages_count,is_released,keywords_count,cast_score,crew_score
count,3000.000000,3.000000e+03,3000.000000,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,0.059635,2.875368e-02,0.319095,4.388234e-02,0.105249,0.510333,0.342667,0.263000,0.247000,0.190333,...,0.000333,0.165792,0.564611,0.485366,0.991429,0.161370,0.998667,0.048501,0.011790,0.082470
std,0.097438,4.112293e-02,0.065323,9.049665e-02,0.241177,0.499977,0.474680,0.440336,0.431339,0.392630,...,0.018257,0.094044,0.284093,0.278483,0.007647,0.098632,0.036497,0.044738,0.034671,0.100740
min,0.000000,3.397466e-09,0.000000,6.580861e-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.083333,0.032258,0.952405,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.365120e-02,0.278107,1.580427e-03,0.002370,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.125000,0.333333,0.258065,0.988101,0.111111,1.000000,0.020134,0.001805,0.016319
50%,0.021053,2.505584e-02,0.307692,1.106050e-02,0.002370,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.125000,0.583333,0.483871,0.993555,0.111111,1.000000,0.040268,0.004711,0.054681
75%,0.078947,3.700174e-02,0.349112,4.526053e-02,0.002370,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.125000,0.833333,0.709677,0.997025,0.222222,1.000000,0.067114,0.010414,0.110075
max,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
test.describe()

,budget,popularity,runtime,name_collection,genre_Drama,genre_Comedy,genre_Thriller,genre_Action,genre_Romance,genre_Adventure,...,prodc_Wonderful Films PLC,prodc_Native Pictures Productions,prodc_Malofilm,prodc_The Image Organization,prodc_Filmtech,prodc_Smokewood Entertainment Group,prodc_UGC DA International,prodc_Pathé Consortium Cinéma,prodc_Les Films de la Pléiade,cast_score
count,4398.000000,4.398000e+03,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,...,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000
mean,0.087032,1.561719e-02,0.336014,0.101438,0.487722,0.358572,0.245566,0.226012,0.196453,0.153934,...,0.000227,0.000227,0.000227,0.000227,0.000227,0.000227,0.000227,0.000227,0.000227,0.038009
std,0.141938,2.230005e-02,0.066554,0.239198,0.499906,0.479636,0.430471,0.418294,0.397360,0.360926,...,0.015079,0.015079,0.015079,0.015079,0.015079,0.015079,0.015079,0.015079,0.015079,0.052093
min,0.000000,1.826523e-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.114646e-03,0.293750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010365
50%,0.028462,1.366649e-02,0.325000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026013
75%,0.107692,1.997947e-02,0.368750,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048130
max,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
y = train.revenue
X = train.drop('revenue', axis=1)

In [11]:
print(train['crew_score'].corr(train['revenue']))

0.8868306222141772


In [14]:
# Select Top 50 Best Features
number_of_features = 50
best_features = SelectKBest(score_func=chi2, k=number_of_features)
y = y.astype('int')
fit = best_features.fit(X, y)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X.columns)
feature_scores = pd.concat([df_columns, df_scores], axis=1)
feature_scores.columns = ['Specs', 'Score']
print(feature_scores.nlargest(number_of_features, 'Score'))

                                             Specs       Score
139                           prodc_Marvel Studios  373.127042
33                        prodc_Paramount Pictures   16.692771
2496                               prodc_Paramount   15.813084
12                           genre_Science Fiction    8.444309
10                                 genre_Adventure    4.981707
7                                     genre_Action    2.296348
24                                    len_homepage    1.487076
4                                      genre_Drama    0.510504
5                                     genre_Comedy    0.342781
6                                   genre_Thriller    0.263088
8                                    genre_Romance    0.190397
9                                      genre_Crime    0.156385
26                                   ortitle_equal    0.117039
11                                    genre_Horror    0.100367
3727                                 release_month    0

In [15]:
selected_features = feature_scores.nlargest(number_of_features, 'Score')['Specs'].tolist()

In [14]:
X = X[selected_features]
X.describe()

NameError: name 'selected_features' is not defined

In [16]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=.2, random_state=13)

In [17]:
print('Training: ', X_tr.shape)
print('Validation: ', X_val.shape)
print('Test: ', test.shape)

Training:  (2400, 3735)
Validation:  (600, 3735)
Test:  (4398, 5046)


In [18]:
def kfold_validate(X, y, te, model, col):
    scores = np.zeros(k)
    for i, (tr_idx, val_idx) in enumerate(cv.split(X)):
        X_tr, y_tr = X.reindex(tr_idx), y.reindex(tr_idx)
        X_val, y_val = X.reindex(val_idx), y.reindex(val_idx)
        
        X_tr.fillna(-1, inplace=True)
        y_tr.fillna(-1, inplace=True)
        X_val.fillna(-1, inplace=True)
        y_val.fillna(-1, inplace=True)
        

        model.fit(X_tr, y_tr)
        pred_val = model.predict(X_val)
        scores[i] = get_loss(pred_val, y_val)
        print("========={}-th Fold Score: {}".format(i, scores[i]))
        
        meta_tr[col][val_idx] = pred_val
        meta_tr['val'][val_idx] = y_val

    print("=========Total Score: ", np.mean(scores))
    model.fit(X, y)
    meta_te[col] = model.predict(te)

In [19]:
def get_loss(pred, actual):
  loss = mean_squared_error(pred, actual)
  result = np.sqrt(loss)
  return result

In [21]:
k = 5
seed = 10
cv = KFold(n_splits = k, shuffle = True, random_state = seed)

In [22]:
meta_tr = pd.DataFrame(np.zeros((X.shape[0], 4)),
                            columns=['catboost', 'random_forest', 'KNN', 'linear_regression'])
meta_tr['val'] = 0
meta_te = pd.DataFrame(np.zeros((test.shape[0], 4)),
                            columns=['catboost', 'random_forest', 'KNN', 'linear_regression'])

### CatBoost

In [23]:
cat_boost = CatBoostRegressor(bagging_temperature = 0.3, loss_function='RMSE', logging_level='Silent',
                             colsample_bylevel = 0.7, depth = 9, early_stopping_rounds = 200, iterations = 1000, eval_metric='RMSE', learning_rate = 0.01)
kfold_validate(X_tr, y_tr, test, cat_boost, 'catboost')

=========0-th Fold Score: 0.045640661499298275


/Users/timluo1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/timluo1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


=========1-th Fold Score: 0.0008360829850115974
=========2-th Fold Score: 0.0010244311857183664
=========3-th Fold Score: 0.000922818611757413
=========4-th Fold Score: 0.0009250002433252542
=========Total Score:  0.00986979890502218


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:79: At position 9 should be feature with name genre_Crime (found genre_Adventure).

In [24]:
cat_boost_pred = cat_boost.predict(X_val)
np.sqrt(mean_squared_error(y_val, cat_boost_pred))

0.00011397590254891602

### Random Forest

In [27]:
random_forest = RandomForestRegressor(n_estimators = 3000, max_depth = 9, criterion='mse')
kfold_validate(X_tr, y_tr, test, random_forest, 'random_forest')

=========0-th Fold Score: 0.04564354645876384


KeyboardInterrupt: 

In [ ]:
random_forest_pred = random_forest.predict(X_val)
np.sqrt(mean_squared_error(y_val, random_forest_pred))

### Linear Regression

In [28]:
linear_reg = LinearRegression()
kfold_validate(X_tr, y_tr, test, linear_reg, 'linear_regression')

=========0-th Fold Score: 0.04562578041607381
=========1-th Fold Score: 38026212.67758469
=========2-th Fold Score: 191324994.51896167
=========3-th Fold Score: 208491613.72703406
=========4-th Fold Score: 152566041.16402355
=========Total Score:  118081772.42664595


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3735 is different from 5046)

In [29]:
lin_res = linear_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, lin_res))

392874980659.62286

### K-NN

In [30]:
knn = KNeighborsRegressor(n_neighbors=10, weights='distance', p=5)
kfold_validate(X_tr, y_tr, test, knn, 'KNN')

=========0-th Fold Score: 0.04564354645876384
=========1-th Fold Score: 0.0044759533037212135
=========2-th Fold Score: 0.007808515501443836
=========3-th Fold Score: 0.008890546310809969
=========4-th Fold Score: 0.004529486879648977
=========Total Score:  0.01426960969087757


ValueError: query data dimension must match training data dimension

In [ ]:
knn.kneighbors_graph()

## Ridge Regression